# 📊 Carga de Datos - Dataset de Fraude Financiero

Este notebook muestra cómo cargar y visualizar inicialmente el dataset de transacciones financieras.

**Nota**: Normalmente en producción, los datos estarían en un Data Warehouse o base de datos,
pero para este ejemplo académico utilizamos un archivo CSV.

## 1. Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Librerías importadas correctamente")

## 2. Cargar el Dataset desde CSV

In [ ]:
# Ruta al archivo CSV
DATA_PATH = "../../financial_fraud_dataset.csv"

# Cargar datos
print(f"📂 Cargando datos desde: {DATA_PATH}")
df = pd.read_csv(DATA_PATH)

print(f"\n✅ Datos cargados exitosamente!")
print(f"   - Filas: {df.shape[0]:,}")
print(f"   - Columnas: {df.shape[1]}")

## 3. Exploración Inicial del Dataset

### 3.1 Primeras Filas

In [ ]:
print("📋 Primeras 10 filas del dataset:\n")
df.head(10)

### 3.2 Información del Dataset

In [ ]:
print("ℹ️ Información del DataFrame:\n")
df.info()

### 3.3 Estadísticas Descriptivas

In [ ]:
print("📊 Estadísticas descriptivas (variables numéricas):\n")
df.describe()

### 3.4 Nombres y Tipos de Columnas

In [ ]:
print("📑 Columnas y sus tipos de datos:\n")
for col, dtype in df.dtypes.items():
    print(f"  • {col:<30} {dtype}")

## 4. Verificación de Valores Nulos

In [ ]:
print("🔍 Valores nulos por columna:\n")
null_counts = df.isnull().sum()
null_pct = (null_counts / len(df)) * 100

null_df = pd.DataFrame({
    'Columna': null_counts.index,
    'Nulos': null_counts.values,
    'Porcentaje': null_pct.values
})

print(null_df.to_string(index=False))

if null_counts.sum() == 0:
    print("\n✅ No hay valores nulos en el dataset")
else:
    print(f"\n⚠️ Se encontraron {null_counts.sum()} valores nulos en total")

## 5. Distribución de la Variable Objetivo

In [ ]:
print("🎯 Distribución de la variable objetivo (is_fraud):\n")

fraud_counts = df['is_fraud'].value_counts()
fraud_pct = df['is_fraud'].value_counts(normalize=True) * 100

print("Conteo:")
print(f"  • No Fraude (0): {fraud_counts[0]:,} ({fraud_pct[0]:.2f}%)")
print(f"  • Fraude (1):    {fraud_counts[1]:,} ({fraud_pct[1]:.2f}%)")
print(f"\n📈 Ratio de desbalanceo: 1:{int(fraud_counts[0]/fraud_counts[1])}")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
fraud_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribución de Transacciones', fontsize=14, fontweight='bold')
axes[0].set_xlabel('is_fraud', fontsize=12)
axes[0].set_ylabel('Cantidad', fontsize=12)
axes[0].set_xticklabels(['No Fraude (0)', 'Fraude (1)'], rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for i, v in enumerate(fraud_counts):
    axes[0].text(i, v + fraud_counts.max()*0.02, f"{v:,}\n({fraud_pct.iloc[i]:.2f}%)", 
                 ha='center', fontweight='bold')

# Gráfico de torta
colors = ['#2ecc71', '#e74c3c']
explode = (0, 0.1)  # Resaltar la clase minoritaria
axes[1].pie(fraud_counts, labels=['No Fraude', 'Fraude'], autopct='%1.2f%%',
            colors=colors, explode=explode, shadow=True, startangle=90)
axes[1].set_title('Proporción de Fraude', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ NOTA: El dataset está altamente DESBALANCEADO.")
print("   Se requiere aplicar técnicas de balanceo (oversampling, undersampling, etc.)")

## 6. Resumen de Columnas por Tipo

In [ ]:
# Identificar columnas irrelevantes (IDs, timestamps)
irrelevant_cols = ['transaction_id', 'timestamp', 'customer_id']

# Identificar columnas numéricas
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'is_fraud' in numerical_cols:
    numerical_cols.remove('is_fraud')  # La variable objetivo no es un feature

# Identificar columnas categóricas
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in irrelevant_cols]

print("📂 Resumen de columnas por tipo:\n")
print(f"Columnas irrelevantes (a eliminar): {irrelevant_cols}")
print(f"\nColumnas numéricas ({len(numerical_cols)}):")
for col in numerical_cols:
    print(f"  • {col}")

print(f"\nColumnas categóricas ({len(categorical_cols)}):")
for col in categorical_cols:
    print(f"  • {col}")

print(f"\nVariable objetivo: is_fraud")

## 7. Muestra de Datos por Categoría

In [ ]:
print("🔍 Valores únicos en variables categóricas:\n")

for col in categorical_cols:
    print(f"\n{col}:")
    print(f"  • Valores únicos: {df[col].nunique()}")
    print(f"  • Categorías: {df[col].unique().tolist()}")
    print(f"\n  Distribución:")
    print(df[col].value_counts().to_string())

## 8. Guardar Dataset Limpio (sin columnas irrelevantes)

In [ ]:
# Eliminar columnas irrelevantes
df_clean = df.drop(columns=irrelevant_cols, errors='ignore')

print(f"📊 Dataset limpio:")
print(f"   - Forma original: {df.shape}")
print(f"   - Forma limpia: {df_clean.shape}")
print(f"   - Columnas eliminadas: {irrelevant_cols}")

# Mostrar primeras filas del dataset limpio
print("\n📋 Primeras filas del dataset limpio:")
df_clean.head()

## 9. Conclusiones de la Carga de Datos

### ✅ Datos Cargados Exitosamente

1. **Tamaño del Dataset**: ~10,000 transacciones
2. **Variables**:
   - 3 columnas numéricas: `amount`, `customer_age`, `previous_transactions`
   - 3 columnas categóricas: `merchant_category`, `customer_location`, `device_type`
   - 1 variable objetivo: `is_fraud`
   
3. **Desbalanceo de Clases**: 
   - El dataset está altamente desbalanceado (~99% no fraude, ~1% fraude)
   - **Acción requerida**: Aplicar técnicas de balanceo durante el entrenamiento

4. **Calidad de Datos**:
   - No se detectaron valores nulos
   - Tipos de datos correctos

### 📝 Próximos Pasos

1. Análisis Exploratorio de Datos (EDA) en `comprension_eda.ipynb`
2. Validación de datos con reglas de negocio
3. Ingeniería de características
4. Entrenamiento de modelos con balanceo de clases